In [1]:
import sys
# from graph_dataset.dataset import GraphDataset
from rgcn_dataset import GraphDataset
from model.tripartite import GNNPolicy
from utilities import *
import torch
import torch_geometric
import os 
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
import gurobipy as gp
import json
from torch_geometric.data import HeteroData

from pytorch_metric_learning import losses
from pytorch_metric_learning.distances import DotProductSimilarity

In [2]:
#set folder
instance = '80_c11_based_8_std'
model = 'tripartite'

# n=int(instance.split('_')[0])*24


model_save_path = f'./model_save/local_search_{instance}_{model}/'
log_save_path =   f"./model_save/local_search_{instance}_{model}_log/"
os.makedirs(model_save_path, exist_ok=True)
os.makedirs(log_save_path, exist_ok=True)

log_file = open(f'{log_save_path}train.log', 'wb')

In [3]:
LEARNING_RATE = 1e-5
NB_EPOCHS =1000
BATCH_SIZE = 8
NUM_WORKERS = 0
ETA_MIN=1e-6

In [4]:
train_data = GraphDataset(f'./datasets/{instance}/small_range_local_search_tripartite/train/')[::6]
valid_data = GraphDataset(f'./datasets/{instance}/small_range_local_search_tripartite/valid/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [5]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
PredictModel = GNNPolicy().to(DEVICE)
# if os.path.exists(model_save_path+'model_best.pth'):
#     state_dict = torch.load(model_save_path+'model_best.pth')
#     PredictModel.load_state_dict(state_dict, strict=False)
#     print("载入模型成功")

In [6]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    infoNCE_loss_function = losses.NTXentLoss(temperature=0.07,distance=DotProductSimilarity()).to(DEVICE)
    # loss_fn = torch.nn.BCELoss()
    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):

            batch = batch.to(DEVICE)
            embeddings = predict(batch).reshape(len(batch), -1)
            
            # print(batch.positive)
            # print(len(batch.positive),len(batch.positive[0]))
            
           
            anchor_positive = torch.tensor([i for i in range(len(batch)) for j in range(len(batch.positive[i]))])
            anchor_negative = torch.tensor([i for i in range(len(batch)) for j in range(len(batch.negative[i]))])
            positive_idx = torch.arange(len(batch),len(batch)+len(anchor_positive))
            negative_idx = torch.arange(len(batch)+len(anchor_positive),len(batch)+len(anchor_positive)+len(anchor_negative))
            
            # print(anchor_positive)
            # print(anchor_negative)
            # print(positive_idx)
            # print(negative_idx)
            positive = torch.from_numpy(np.concatenate(batch.positive)).to(DEVICE)
            negative = torch.from_numpy(np.concatenate(batch.negative)).to(DEVICE)
            # positive_idx = range(1,1+len(batch.positive))
            # negative_idx = range(1+len(batch.positive),1+len(batch.positive)+len(batch.negative))
            embeddings = torch.cat((embeddings, positive, negative),0)
            # print(embeddings.shape)
            triplets = (anchor_positive.to(DEVICE), positive_idx.to(DEVICE), anchor_negative.to(DEVICE), negative_idx.to(DEVICE))
            # triplets = (anchor_positive.clone().detach().to(DEVICE), positive_idx.clone().detach().to(DEVICE), anchor_negative.clone().detach().to(DEVICE), negative_idx.clone().detach().to(DEVICE))
            
            # break
            loss = infoNCE_loss_function(embeddings, indices_tuple = triplets)
            # loss = loss_fn(embeddings, average_local)
            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed
    return mean_loss


In [7]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS//10, eta_min=ETA_MIN)
best_val_loss = 10
cur_epoch = 0


In [9]:
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.8f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.8f}")
    if valid_loss<best_val_loss:
        cur_epoch=0
        best_val_loss = valid_loss
        print("best_loss",best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best_con.pth')
    if cur_epoch%2==0:
        torch.save(PredictModel.state_dict(),model_save_path+f'model_{cur_epoch}.pth')
#     if optloss<best_optloss:
#         best_optloss=optloss
#         print("best_optloss",best_optloss)
#         torch.save(PredictModel.state_dict(),model_save_path+'model_best_acc.pth')
#     torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
#     #100个epochloss不下降就停止
    if cur_epoch>100:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 1.27756768	Epoch 0 Valid loss: 1.22714009
Epoch 1 Train loss: 1.27733321	Epoch 1 Valid loss: 1.22293805
Epoch 2 Train loss: 1.28381844	Epoch 2 Valid loss: 1.20092567
Epoch 3 Train loss: 1.29003629	Epoch 3 Valid loss: 1.23121278
Epoch 4 Train loss: 1.28879330	Epoch 4 Valid loss: 1.21329782
Epoch 5 Train loss: 1.29081063	Epoch 5 Valid loss: 1.20158853
Epoch 6 Train loss: 1.28491730	Epoch 6 Valid loss: 1.24862767
Epoch 7 Train loss: 1.29054021	Epoch 7 Valid loss: 1.22218348
Epoch 8 Train loss: 1.30430916	Epoch 8 Valid loss: 1.25890241
Epoch 9 Train loss: 1.29665060	Epoch 9 Valid loss: 1.20860712
Epoch 10 Train loss: 1.28914816	Epoch 10 Valid loss: 1.23204415
Epoch 11 Train loss: 1.28803452	Epoch 11 Valid loss: 1.24399421
Epoch 12 Train loss: 1.28854201	Epoch 12 Valid loss: 1.20884803
Epoch 13 Train loss: 1.28008031	Epoch 13 Valid loss: 1.24467414
Epoch 14 Train loss: 1.28944060	Epoch 14 Valid loss: 1.32472888
Epoch 15 Train loss: 1.30191021	Epoch 15 Valid loss: 1.27362